In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import seaborn as sns
%matplotlib inline
import json
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.linear_model import SGDClassifier
from collections import Counter
import nlpaug.augmenter.word as naw
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score

In [3]:
nltk.download('wordnet')

<IPython.core.display.Javascript object>

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
sklearn.__version__

<IPython.core.display.Javascript object>

'1.0.2'

In [20]:
df = pd.read_csv("hi.csv")

In [21]:
df

,question,answer
0,hi,Hello! How can I assist you today?
1,how are you,Im doing well thank you. Is there anything you...
2,good morning,Good morning to you too! How can I assist you ...
3,good afternoon,Good afternoon to you too! How can I assist yo...
4,good evening,Good evening to you too! How can I assist you ...
...,...,...
333,what is the minimum attendance percentage requ...,80%.
334,photo,https://nmims.edu/about/photogallery-shirpur-c...
335,photo,https://nmims.edu/about/photogallery-shirpur-c...
336,gallery,https://nmims.edu/about/photogallery-shirpur-c...


In [22]:
# lemmatization on question
lemmatizer = WordNetLemmatizer()
def data(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(text)
    return text


In [23]:
df["question"] = df["question"].apply(data) 

In [24]:
df["question"]

0                                                     hi
1                                            how are you
2                                           good morning
3                                         good afternoon
4                                           good evening
                             ...                        
333    what is the minimum attendance percentage requ...
334                                                photo
335                                                photo
336                                              gallery
337                                        photo gallery
Name: question, Length: 338, dtype: object

In [25]:
import pandas as pd
import nlpaug.augmenter.word as naw

# Load the DataFrame

# Initialize the synonym replacement augmenter
synonym_aug = naw.SynonymAug(aug_src='wordnet')

# Augment the "question" column
augmented_questions = []
for question in df["question"]:
    # Perform sentence shuffling
    shuffled_question = naw.RandomWordAug(action="swap").augment(question)
    
    augmented_questions.append(shuffled_question)

    # Perform synonym replacement
#     synonym_replaced_questions = [synonym_aug.augment(shuffled_question)]
    
#     augmented_questions.extend(synonym_replaced_questions)
# augmented_questions
# # Create a new DataFrame with the augmented questions and corresponding answers
new_df = pd.DataFrame({"question": augmented_questions, "answer": df["answer"]})
df = df.append(new_df)
# # Save the new DataFrame to an Excel file
df.to_excel("df_shuff.xlsx", index=False)
df

C:\Users\priya\AppData\Local\Temp\ipykernel_12684\4024978548.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


,question,answer
0,hi,Hello! How can I assist you today?
1,how are you,Im doing well thank you. Is there anything you...
2,good morning,Good morning to you too! How can I assist you ...
3,good afternoon,Good afternoon to you too! How can I assist yo...
4,good evening,Good evening to you too! How can I assist you ...
...,...,...
333,[the what is minimum attendance percentage for...,80%.
334,[photo],https://nmims.edu/about/photogallery-shirpur-c...
335,[photo],https://nmims.edu/about/photogallery-shirpur-c...
336,[gallery],https://nmims.edu/about/photogallery-shirpur-c...


In [26]:
import pandas as pd
import nltk
from nltk.corpus import wordnet

# Load the DataFrame
df_shuff = pd.read_csv("df_shuff.csv")  # Replace "your_data.csv" with the actual filename or path

# Download the WordNet corpus if not already downloaded

# Function to replace words with their synonyms
def replace_with_synonyms(sentence):
    words = nltk.word_tokenize(sentence)
    synonyms = []
    for word in words:
        synsets = wordnet.synsets(word)
        if synsets:
            synonyms.append(synsets[0].lemmas()[0].name())
        else:
            synonyms.append(word)
    return ' '.join(synonyms)

# Augment the "question" column
augmented_questions = df_shuff["question"].apply(replace_with_synonyms)

# Create a new DataFrame with the augmented questions and corresponding answers
new_df = pd.DataFrame({"question": augmented_questions, "answer": df_shuff["answer"]})
df_shuff = df_shuff.append(new_df)
df_shuff
# Save the new DataFrame to an Excel file
df_shuff.to_excel("df_shuff.xlsx", index=False)


C:\Users\priya\AppData\Local\Temp\ipykernel_12684\932250850.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shuff = df_shuff.append(new_df)


In [9]:
# # aug_data = {}
# # for ques,ans in tqdm(zip(df['question'],df['answer'])):
# #     for i in range(4):
# #         aug_data.update({aug.augment(ques):ans})
 
    
# aug = naw.WordEmbsAug(model_type='glove', model_path="./glove.6B.100d.txt",action="substitute")

# augmented_data = []

# for ques, ans in tqdm(zip(df['question'], df['answer'])):
#     augmented_questions = [aug.augment(ques) for _ in range(4)]
#     for augmented_question in augmented_questions:
#         augmented_data.append((augmented_question, ans))

# # Create a new DataFrame from the augmented data
# augmented_df = pd.DataFrame(augmented_data, columns=['question', 'answer'])


In [10]:
aug_df = pd.DataFrame(augmented_data, columns=['question', 'answer'])
aug_df

NameError: name 'augmented_data' is not defined

In [17]:
final_df = pd.concat([df,aug_df])

In [18]:
final_df.to_csv("augmented.csv",index=False)

In [19]:
aug_df.to_csv("aug.csv",index=False)

In [20]:
df.to_csv("df.csv",index=False)

In [20]:
aug_dict = {item['question']: item['answer'] for item in augmented_data}

print(aug_dict)

TypeError: tuple indices must be integers or slices, not str

In [21]:
old_data_dict = {item['question']: item['answer'] for item in df}

TypeError: string indices must be integers

In [22]:
new_data_dict = {augmented_data}

TypeError: unhashable type: 'list'

In [29]:
# for i in range(0, len(augmented_data)):
print(augmented_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
wordnet=WordNetLemmatizer()
corpus1 = []
for i1 in range(0, len(df)):
    review1 = re.sub('[^a-zA-Z]', ' ', df['question'][i1])
    review1 = review1.lower()
    review1 = review1.split()
    
    review1 = [wordnet.lemmatize(word) for word in review1 if not word in stopwords.words('english')]
    review1 = ' '.join(review1)
    corpus1.append(review1)

In [9]:
# corpus1

['change password',
 'receive changed atm pin',
 'get newly generated pin online',
 'register autopay',
 'chip credit card used anywhere',
 'information required use e monies neft service',
 'money credited beneficiary account using e monies neft service get back',
 'use e monies neft foreign remittance',
 'ifsc different micr code',
 'charge e monies national electronic fund transfer service',
 'e monies neft service differ rgts eft',
 'e monies neft system better existing rbi eft system',
 'check ifsc receiving branch',
 'information need provide using rtgs fund transfer service',
 'money credited successfully remitting customer get fund back',
 'long take fund transferred one account another',
 'minimum maximum limit rtgs transfer',
 'rtgs different national electronic fund transfer neft',
 'rtgs fund transfer',
 'charge rtgs fund transfer service',
 'customer required fill cardholder dispute form cdf report transaction dispute',
 'time frame report credit card transaction dispute',

In [10]:
# wordnet=WordNetLemmatizer()

# corpus2 = []
# for i2 in range(0, len(df)):
#     review2 = re.sub('[^a-zA-Z]', ' ', df['answer'][i2])
#     review2 = review2.lower()
#     review2 = review2.split()
#     review2 = [wordnet.lemmatize(word) for word in review2 if not word in stopwords.words('english')]

#     review2 = ' '.join(review2)
#     corpus2.append(review2)

In [11]:
# corpus2

['logged change password using change password option top part screen type current password new password chosen respective box',
 'receive new atm pin post within day request submitted',
 'security reason send atm pin post',
 'register autopay step click autopay register link left side margin step select credit card number want register autopay facility hdfc bank account number want credit card payment made step want full amount statement paid bank account select total amount due link want minimum amount due total amount paid select minimum amount due step click continue confirm message displayed screen confirming successful registration credit card autopay please note take day activate autopay facility credit card account payment due date day le away date registration autopay kindly pay monthly bill usual payment mode autopay effective next billing cycle',
 'yes hdfc bank chip credit card used anywhere world wherever visa mastercard accepted chip enabled terminal insert chip card po t

In [14]:
# Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Training model using Naive bayes classifier

# from sklearn.naive_bayes import MultinomialNB
# spam_detect_model = MultinomialNB().fit(X_train, y_train)

# y_pred=spam_detect_model.predict(X_test)

In [15]:
# # y_train = y_train.ravel()

# # Create an SVM object with hyperparameters of your choice
# svm_model = svm.SVC(kernel='linear', C=1, gamma='scale')

# # Train the model on the training dataset
# svm_model.fit(X_train, y_train)
# #
# # Make predictions on the test dataset
# y_pred = svm_model.predict(X_test)

# # Evaluate the model performance

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy of SVM model: {accuracy}")


In [136]:
# # Check the shape of X_train and y_train
# print('X_train shape:', X_train.shape)
# print('y_train shape:', y_train.shape)

# # Check the values of X_train and y_train
# print('X_train values:', X_train)
# print('y_train values:', y_train)



X_train shape: (1788, 1587)
y_train shape: (1788, 5514)
X_train values: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y_train values: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
